# Web Scraping Project
## import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import bs4
import pandas as pd
import time


In [2]:
url_L=[]
name_L=[]
price_L=[]
sold_L=[]
amount_reviewer_L=[]
all_store_reviewer_L=[]
all_product_store_L=[]
response_rate_L=[]
follower_L=[]
joined_L=[]
rating_L=[]
new_url_L=[]

## setup the driver

In [3]:
driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options()) #choose the driver ex.Edge, Firefox
link=f"https://shopee.co.th" #insert the shopee url
driver.get(link)
driver.maximize_window()

In [4]:
#change language to english in shopee
WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH,f"/html/body/div[2]/div[1]/div[1]/div/div[3]/div[2]/button")))
links=driver.find_element(By.XPATH,f"/html/body/div[2]/div[1]/div[1]/div/div[3]/div[2]/button")
links.click()

In [5]:
#close the ads
ads = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
ads.click()

In [6]:
catergories = driver.find_element(By.XPATH,f'/html/body/div[1]/div/div[2]/div/div/div[3]/div[3]/div[1]/div/div[2]/div/div[1]/ul/li[20]/a')
catergories.click()

In [7]:
driver.execute_script("document.body.style.zoom = '15%'")

## Functions

In [8]:
wait=WebDriverWait(driver, 10)
def k_and_m_to_float(txt):
    "ฟังก์ชันเพื่อเปลี่ยน k เป็น 1000 เปลี่ยน m เป็น 1000000 เปลี่ยนเป็น float, this function change k to multiple 100 and m to multiple 1000000 "
    if txt[-1]=="k":
        txt=(float(txt[:-1])*1000)
    elif txt[-1]=="m":
        txt=(float(txt[:-1])*1000000)
    else:
        txt=(float(txt))
    return txt

def hours_convert(txt):
    "ฟังก์ชันเพื่อเปลี่ยนทุกหน่วยเป็นหน่วยชั่วโมง,change, this function change all time unit in to hours"
    if txt[-10:]==' years ago':
        txt=float(txt[:-10])*360*24
    elif txt[-10:]==' hours ago':
        txt=float(txt[:-10])
    elif txt[-11:]==' months ago':
        txt=float(txt[:-11])*30*24
    elif txt[-9:]==' days ago':
        txt=float(txt[:-9])*24
    else:
        txt=(float(0))
    return txt

def waiter(datatype,positiondata):
    "datatype ex. By.CSS_SELECTOR ,positiondata ex. f'.pmmxKx'"
    wait.until(EC.visibility_of_element_located((datatype,positiondata)))
    global soup
    soup=bs4.BeautifulSoup(driver.page_source)

## Url archive page

In [11]:
for N_of_page in range(1,9): #total of page 
    link=f"https://shopee.co.th/Gaming-Accessories-cat.11044961?page={N_of_page}" #insert the shopee url
    driver.get(link)
    driver.execute_script('document.body.style.zoom="15%"')
    try:
        time.sleep(3)
        soup=bs4.BeautifulSoup(driver.page_source)
        url=soup.find_all('div', {'class':'col-xs-2-4 shopee-search-item-result__item'})
    except:
        print("not found a product row")
    for i in url:
        url_L.append(i.find('a').get('href'))
        print(len(url_L))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


AttributeError: 'NoneType' object has no attribute 'get'

In [10]:
print(url_L)

[]


## Scrape Feature(X)

In [14]:
for i in range ((len(url_L))):
    w=('https://shopee.co.th'+url_L[i])
    new_url_L.append(w)
    print(len(new_url_L))
    driver.get(w)
    driver.execute_script('document.body.style.zoom="1%"')    
    soup=bs4.BeautifulSoup(driver.page_source)
    #ชื่อสินค้า Name
    try:
        waiter(By.CSS_SELECTOR,".VCNVHn")
        name_L.append(soup.find('div',{'class':'VCNVHn'}).text)
    except:
        name_L.append("None")
    #ราคา Price
    try:
        waiter(By.CSS_SELECTOR,f".pmmxKx")
        try:
            price_L.append(float(soup.find('div',{'class':'pmmxKx'}).text.replace(",",'').replace("฿","")))
        except:
            price_L.append(float((soup.find('div',{'class':'pmmxKx'}).text.replace(",",'').replace("฿","")).partition(' - ')[0]))
    except:
        price_L.append("None")
    #จำนวนที่ขาย sold
    try:
        waiter(By.CSS_SELECTOR,f"._45NQT5")
        sold=(soup.find('div',{'class':'_45NQT5'}).text)
        sold_L.append(k_and_m_to_float(sold))
    except:
        sold_L.append('None')
    #จำนวนคนรีวิว amount reviewer
    try:
        waiter(By.CSS_SELECTOR,f"div.HXGLyo:nth-child(2) > div:nth-child(1)")
        amount_reviewer=(((soup.find_all('div',{'class':'MrYJVA'}))[1]).text)
        amount_reviewer_L.append(k_and_m_to_float(amount_reviewer))
    except:
        amount_reviewer_L.append(float(0))
    #จำนวนคนรีวิวร้านค้าทั้งหมด all store reviewer
    try:
        waiter(By.CSS_SELECTOR,f"div.MSPJbO:nth-child(1) > div:nth-child(1) > span:nth-child(2)")
        all_store_reviewer=((soup.find_all('span',{'class':'_32ZDbL'}))[0].text)
        all_store_reviewer_L.append(k_and_m_to_float(all_store_reviewer))
    except:
        all_store_reviewer_L.append('None')
    #จำนวนสินค้าทั้งหมดของร้านค้า all product of store
    try:
        waiter(By.CSS_SELECTOR,f".g54jiy")
        all_product_store_L.append(k_and_m_to_float((soup.find('span',{'class':'_32ZDbL g54jiy'}).text)))
    except:
        all_product_store_L.append('None')
    #อัตราการตอบกลับของร้านค้า response rate of store
    try:
        waiter(By.CSS_SELECTOR,f"div.MSPJbO:nth-child(2) > div:nth-child(1) > span:nth-child(2)")
        response_rate_L.append(float((soup.find_all('span',{'class':'_32ZDbL'}))[2].text.replace('%','')))
    except:
        response_rate_L.append('None')
    #จำนวนผู้ติดตาม follower of store
    try:
        waiter(By.CSS_SELECTOR,f"div.MSPJbO:nth-child(3) > div:nth-child(2) > span:nth-child(2)")
        follower_L.append(k_and_m_to_float((soup.find_all('span',{'class':'_32ZDbL'}))[5].text))
    except:
        follower_L.append('None')
    #เข้าร่วมเมื่อกี่ชั่วโมงแล้ว joined
    try:
        waiter(By.CSS_SELECTOR,f"div.pHNb7U:nth-child(1) > span:nth-child(2)")
        joined_L.append(hours_convert((soup.find_all('span',{'class':'_32ZDbL'}))[4].text))
    except:  
        joined_L.append('None')
    #rating
    try:
        waiter(By.CSS_SELECTOR,f".Ga-lTj")
        rating_L.append(float((soup.find('div',{'class':'MrYJVA Ga-lTj'})).text))
    except:
        rating_L.append(float(0))

## Make to dataframe for use

In [22]:
shopee=pd.DataFrame([new_url_L,name_L,price_L,sold_L,amount_reviewer_L,all_store_reviewer_L,all_product_store_L,response_rate_L,follower_L,joined_L,rating_L]).transpose()
shopee.columns=['url','name','price','sold','amount reviewer','all reviewer of store','amount product','response rate','follower','joined','rating']
shopee.to_csv(r'/Users/thanapatpiyamssakul/coding/first_year/midterm/test.csv')